# Carregando Dados de Teste para MongoDB

Este notebook insere dados mockados no MongoDB para testar a API de filmes.
Ele vai criar filmes, usuários e avaliações para fins de teste.

bibliotecas necessárias: pip install motor faker pandas jupyter notebook

## Configuração e Dependências

Primeiro, vamos importar as bibliotecas necessárias e configurar a conexão com o MongoDB.

In [1]:
import asyncio
import os
from datetime import datetime, timedelta
from bson import ObjectId
from motor.motor_asyncio import AsyncIOMotorClient
from faker import Faker
import random
import pandas as pd
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

# Criar instância do Faker para gerar dados aleatórios
fake = Faker('pt_BR')

# Configuração do MongoDB
MONGO_URL = "mongodb://localhost:27017"  # Altere para a URL do seu container
DATABASE_NAME = "myfastapidb"

print(f"Conectando a {MONGO_URL}...")

Conectando a mongodb://localhost:27017...


## Função para Conectar ao MongoDB

Esta função estabelece a conexão com o MongoDB usando o driver assíncrono Motor.

In [2]:
async def connect_to_mongo():
    """Conectar ao MongoDB e retornar o cliente e o banco de dados"""
    client = AsyncIOMotorClient(MONGO_URL)
    db = client[DATABASE_NAME]
    return client, db

# Função para fechar a conexão
async def close_mongo_connection(client):
    """Fechar a conexão com o MongoDB"""
    client.close()

## Limpeza do Banco de Dados

Limpar o banco de dados antes de inserir os novos dados de teste.

In [3]:
async def clean_database(db):
    """Limpar todas as coleções do banco de dados"""
    collections = ['movies', 'users', 'reviews']
    
    for collection in collections:
        await db[collection].delete_many({})
        print(f"Coleção {collection} limpa com sucesso.")

# Executar limpeza
async def run_cleanup():
    client, db = await connect_to_mongo()
    await clean_database(db)
    await close_mongo_connection(client)
    
# Executar de forma assíncrona
await run_cleanup()

Coleção movies limpa com sucesso.
Coleção users limpa com sucesso.
Coleção reviews limpa com sucesso.


## Criação de Dados de Filmes

Vamos criar dados para 50 filmes com informações realistas.

In [4]:
def generate_movies(count=50):
    """Gerar dados de filmes para teste"""
    # Dados para aumentar o realismo
    genres = [
        "Ação", "Aventura", "Animação", "Biografia", "Comédia", 
        "Crime", "Documentário", "Drama", "Família", "Fantasia", 
        "História", "Terror", "Musical", "Mistério", "Romance", 
        "Ficção Científica", "Thriller", "Guerra", "Faroeste"
    ]
    
    famous_directors = [
        "Steven Spielberg", "Martin Scorsese", "Christopher Nolan",
        "Quentin Tarantino", "James Cameron", "Ridley Scott",
        "Francis Ford Coppola", "Peter Jackson", "David Fincher",
        "Woody Allen", "Tim Burton", "Stanley Kubrick",
        "Alfred Hitchcock", "Clint Eastwood", "Spike Lee"
    ]
    
    famous_actors = [
        "Tom Hanks", "Leonardo DiCaprio", "Robert De Niro", "Meryl Streep",
        "Brad Pitt", "Johnny Depp", "Denzel Washington", "Morgan Freeman",
        "Jennifer Lawrence", "Sandra Bullock", "Scarlett Johansson", "Emma Stone",
        "Viola Davis", "Samuel L. Jackson", "Harrison Ford", "Tom Cruise",
        "Will Smith", "Keanu Reeves", "Anthony Hopkins", "Al Pacino",
        "Anne Hathaway", "Cate Blanchett", "Angelina Jolie", "Daniel Day-Lewis"
    ]
    
    # Lista de filmes conhecidos para adicionar à lista
    known_movies = [
        {
            "_id": ObjectId("60d21b4967d0d8992e610c86"),
            "title": "The Shawshank Redemption",
            "genres": ["Drama"],
            "director": "Frank Darabont",
            "actors": ["Tim Robbins", "Morgan Freeman", "Bob Gunton"]
        },
        {
            "_id": ObjectId("60d21b4967d0d8992e610c87"),
            "title": "The Godfather",
            "genres": ["Crime", "Drama"],
            "director": "Francis Ford Coppola",
            "actors": ["Marlon Brando", "Al Pacino", "James Caan"]
        },
        {
            "_id": ObjectId("60d21b4967d0d8992e610c88"),
            "title": "The Dark Knight",
            "genres": ["Action", "Crime", "Drama"],
            "director": "Christopher Nolan",
            "actors": ["Christian Bale", "Heath Ledger", "Aaron Eckhart"]
        },
        {
            "_id": ObjectId("60d21b4967d0d8992e610c89"),
            "title": "Inception",
            "genres": ["Action", "Adventure", "Sci-Fi"],
            "director": "Christopher Nolan",
            "actors": ["Leonardo DiCaprio", "Joseph Gordon-Levitt", "Ellen Page"]
        },
        {
            "_id": ObjectId("60d21b4967d0d8992e610c8a"),
            "title": "Pulp Fiction",
            "genres": ["Crime", "Drama"],
            "director": "Quentin Tarantino",
            "actors": ["John Travolta", "Uma Thurman", "Samuel L. Jackson"]
        }
    ]
    
    # Começar com os filmes conhecidos
    movies = known_movies.copy()
    
    # Continuar a gerar até atingir o número desejado
    for i in range(len(movies), count):
        # Gerar um filme aleatório
        movie = {
            "_id": ObjectId(),
            "title": fake.catch_phrase(),
            "genres": random.sample(genres, random.randint(1, 3)),
            "director": random.choice(famous_directors),
            "actors": random.sample(famous_actors, random.randint(3, 6))
        }
        movies.append(movie)
    
    return movies

# Gerar filmes
movies_data = generate_movies()

# Mostrar alguns filmes de exemplo
for movie in movies_data[:3]:
    pprint(movie)

{'_id': ObjectId('60d21b4967d0d8992e610c86'),
 'actors': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'],
 'director': 'Frank Darabont',
 'genres': ['Drama'],
 'title': 'The Shawshank Redemption'}
{'_id': ObjectId('60d21b4967d0d8992e610c87'),
 'actors': ['Marlon Brando', 'Al Pacino', 'James Caan'],
 'director': 'Francis Ford Coppola',
 'genres': ['Crime', 'Drama'],
 'title': 'The Godfather'}
{'_id': ObjectId('60d21b4967d0d8992e610c88'),
 'actors': ['Christian Bale', 'Heath Ledger', 'Aaron Eckhart'],
 'director': 'Christopher Nolan',
 'genres': ['Action', 'Crime', 'Drama'],
 'title': 'The Dark Knight'}


## Criação de Dados de Usuários

Vamos criar alguns usuários para testes, incluindo seus dados e senhas já hasheadas.

In [5]:
def generate_users(count=10):
    """Gerar dados de usuários para teste"""
    # Hash de senha mockado - em produção usaria bcrypt
    hashed_password = "$2b$12$EixZaYVK1fsbw1ZfbX3OXePaWxn96p36WQoeG6Lruj3vjPGga31lW"  # 'password'
    
    # Começar com os usuários de teste conhecidos
    users = [
        {
            "_id": ObjectId("60d21b4967d0d8992e610c85"),
            "username": "testuser",
            "email": "test@example.com",
            "password_hash": hashed_password,
            "created_at": datetime.utcnow() - timedelta(days=30)
        },
        {
            "_id": ObjectId("60d21b4967d0d8992e610c84"),
            "username": "emptyuser",
            "email": "empty@example.com",
            "password_hash": hashed_password,
            "created_at": datetime.utcnow() - timedelta(days=20)
        }
    ]
    
    # Gerar usuários adicionais
    for i in range(len(users), count):
        user = {
            "_id": ObjectId(),
            "username": fake.user_name(),
            "email": fake.email(),
            "password_hash": hashed_password,
            "created_at": datetime.utcnow() - timedelta(days=random.randint(1, 60))
        }
        users.append(user)
    
    return users

# Gerar usuários
users_data = generate_users()

# Mostrar alguns usuários de exemplo
for user in users_data[:3]:
    print(f"Username: {user['username']}, Email: {user['email']}")

Username: testuser, Email: test@example.com
Username: emptyuser, Email: empty@example.com
Username: maria-flor83, Email: isouza@example.net


## Criação de Dados de Avaliações

Agora vamos criar avaliações de filmes pelos usuários.

In [6]:
def generate_reviews(users, movies, count=100):
    """Gerar dados de avaliações para teste"""
    # Comentários comuns para avaliações
    positive_comments = [
        "Excelente filme! Recomendo muito.",
        "Um dos melhores que já assisti.",
        "Ótima atuação e direção impecável.",
        "Roteiro surpreendente e bem executado.",
        "Adorei cada minuto, muito envolvente!"
    ]
    
    neutral_comments = [
        "Filme razoável, vale assistir uma vez.",
        "Teve bons momentos, mas também fracos.",
        "Esperava mais, mas não foi ruim.",
        "Uma história interessante, mas previsível.",
        "Entretenimento médio."
    ]
    
    negative_comments = [
        "Não recomendo, muito fraco.",
        "Desperdiçou um bom potencial.",
        "Roteiro confuso e atuações fracas.",
        "Faltou desenvolvimento dos personagens.",
        "Muito longo e pouco interessante."
    ]
    
    # Começar com as avaliações conhecidas dos testes
    reviews = [
        {
            "_id": ObjectId("60d21b4967d0d8992e610c90"),
            "user_id": str(users[0]["_id"]),  # testuser
            "movie_id": str(movies[0]["_id"]),  # Shawshank
            "rating": 5.0,
            "comment": "Excelente filme!",
            "created_at": datetime.utcnow() - timedelta(days=15)
        },
        {
            "_id": ObjectId("60d21b4967d0d8992e610c91"),
            "user_id": str(users[0]["_id"]),  # testuser
            "movie_id": str(movies[1]["_id"]),  # Godfather
            "rating": 4.5,
            "comment": "Um clássico!",
            "created_at": datetime.utcnow() - timedelta(days=10)
        }
    ]
    
    # Manter registro de pares usuário-filme já avaliados
    evaluated_pairs = set([(review["user_id"], review["movie_id"]) for review in reviews])
    
    # Gerar avaliações adicionais aleatórias
    while len(reviews) < count:
        user = random.choice(users)
        movie = random.choice(movies)
        
        user_id = str(user["_id"])
        movie_id = str(movie["_id"])
        
        # Evitar duplicatas
        if (user_id, movie_id) in evaluated_pairs:
            continue
        
        # Gerar avaliação aleatória
        rating = round(random.uniform(1.0, 5.0) * 2) / 2  # Arredondar para incrementos de 0.5
        
        # Selecionar comentário baseado na avaliação
        if rating >= 4.0:
            comment = random.choice(positive_comments)
        elif rating >= 2.5:
            comment = random.choice(neutral_comments)
        else:
            comment = random.choice(negative_comments)
        
        # Criar avaliação
        review = {
            "_id": ObjectId(),
            "user_id": user_id,
            "movie_id": movie_id,
            "rating": rating,
            "comment": comment,
            "created_at": datetime.utcnow() - timedelta(days=random.randint(1, 30))
        }
        
        # Adicionar à lista e ao conjunto de pares avaliados
        reviews.append(review)
        evaluated_pairs.add((user_id, movie_id))
    
    return reviews

# Gerar avaliações
reviews_data = generate_reviews(users_data, movies_data)

# Mostrar distribuição das avaliações
ratings = [review["rating"] for review in reviews_data]
average_rating = sum(ratings) / len(ratings)

print(f"Total de avaliações geradas: {len(reviews_data)}")
print(f"Avaliação média: {average_rating:.2f}")

# Mostrar algumas avaliações de exemplo
for review in reviews_data[:3]:
    print(f"Usuário: {review['user_id']}, Filme: {review['movie_id']}, Nota: {review['rating']}")
    print(f"Comentário: {review['comment']}\n")

Total de avaliações geradas: 100
Avaliação média: 3.03
Usuário: 60d21b4967d0d8992e610c85, Filme: 60d21b4967d0d8992e610c86, Nota: 5.0
Comentário: Excelente filme!

Usuário: 60d21b4967d0d8992e610c85, Filme: 60d21b4967d0d8992e610c87, Nota: 4.5
Comentário: Um clássico!

Usuário: 67f29e314e8a747725e0c8b3, Filme: 67f29e314e8a747725e0c89c, Nota: 3.0
Comentário: Teve bons momentos, mas também fracos.



## Inserir Dados no MongoDB

Agora vamos inserir todos os dados no MongoDB.

In [7]:
async def insert_data_to_mongo():
    """Inserir todos os dados no MongoDB"""
    client, db = await connect_to_mongo()
    
    try:
        # Inserir filmes
        result_movies = await db.movies.insert_many(movies_data)
        print(f"Inseridos {len(result_movies.inserted_ids)} filmes no banco de dados.")
        
        # Inserir usuários
        result_users = await db.users.insert_many(users_data)
        print(f"Inseridos {len(result_users.inserted_ids)} usuários no banco de dados.")
        
        # Inserir avaliações
        result_reviews = await db.reviews.insert_many(reviews_data)
        print(f"Inseridas {len(result_reviews.inserted_ids)} avaliações no banco de dados.")
        
        print("\nTodos os dados foram inseridos com sucesso!")
    except Exception as e:
        print(f"Erro ao inserir dados: {str(e)}")
    finally:
        await close_mongo_connection(client)

# Executar inserção de dados
await insert_data_to_mongo()

Inseridos 50 filmes no banco de dados.
Inseridos 10 usuários no banco de dados.
Inseridas 100 avaliações no banco de dados.

Todos os dados foram inseridos com sucesso!


## Verificar os Dados Inseridos

Vamos verificar os dados inseridos no MongoDB.

In [8]:
async def verify_data():
    """Verificar os dados inseridos no MongoDB"""
    client, db = await connect_to_mongo()
    
    try:
        # Contar documentos em cada coleção
        movies_count = await db.movies.count_documents({})
        users_count = await db.users.count_documents({})
        reviews_count = await db.reviews.count_documents({})
        
        print(f"Filmes: {movies_count}")
        print(f"Usuários: {users_count}")
        print(f"Avaliações: {reviews_count}")
        
        print("\nDados verificados com sucesso.")
        
        # Buscar alguns exemplos
        print("\nExemplo de filme:")
        movie = await db.movies.find_one({})
        pprint(movie)
        
        print("\nExemplo de usuário:")
        user = await db.users.find_one({})
        user_display = {k:v for k,v in user.items() if k != 'password_hash'}
        pprint(user_display)
        
        print("\nExemplo de avaliação:")
        review = await db.reviews.find_one({})
        pprint(review)
    except Exception as e:
        print(f"Erro ao verificar dados: {str(e)}")
    finally:
        await close_mongo_connection(client)

# Executar verificação
await verify_data()

Filmes: 50
Usuários: 10
Avaliações: 100

Dados verificados com sucesso.

Exemplo de filme:
{'_id': ObjectId('60d21b4967d0d8992e610c86'),
 'actors': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'],
 'director': 'Frank Darabont',
 'genres': ['Drama'],
 'title': 'The Shawshank Redemption'}

Exemplo de usuário:
{'_id': ObjectId('60d21b4967d0d8992e610c85'),
 'created_at': datetime.datetime(2025, 3, 7, 15, 30, 57, 440000),
 'email': 'test@example.com',
 'username': 'testuser'}

Exemplo de avaliação:
{'_id': ObjectId('60d21b4967d0d8992e610c90'),
 'comment': 'Excelente filme!',
 'created_at': datetime.datetime(2025, 3, 22, 15, 30, 57, 459000),
 'movie_id': '60d21b4967d0d8992e610c86',
 'rating': 5.0,
 'user_id': '60d21b4967d0d8992e610c85'}


## Consultas Avançadas

Vamos realizar algumas consultas avançadas para verificar os dados.

In [9]:
async def advanced_queries():
    """Realizar consultas avançadas no MongoDB"""
    client, db = await connect_to_mongo()
    
    try:
        # 1. Filmes com maior média de avaliação (usando aggregation)
        print("Top 5 filmes com melhor avaliação:")
        pipeline = [
            {
                "$lookup": {
                    "from": "reviews",
                    "localField": "_id",
                    "foreignField": "movie_id",
                    "as": "reviews"
                }
            },
            {
                "$match": {
                    "reviews": {"$ne": []}
                }
            },
            {
                "$addFields": {
                    "avg_rating": {"$avg": "$reviews.rating"},
                    "review_count": {"$size": "$reviews"}
                }
            },
            {
                "$sort": {"avg_rating": -1}
            },
            {
                "$limit": 5
            },
            {
                "$project": {
                    "_id": 1,
                    "title": 1,
                    "avg_rating": 1,
                    "review_count": 1
                }
            }
        ]
        
        cursor = db.movies.aggregate(pipeline)
        results = await cursor.to_list(length=5)
        
        for movie in results:
            print(f"Título: {movie['title']}")
            print(f"Avaliação média: {movie['avg_rating']:.2f} ({movie['review_count']} avaliações)")
            print()
        
        # 2. Usuários mais ativos (com mais avaliações)
        print("\nTop 3 usuários mais ativos:")
        pipeline = [
            {
                "$group": {
                    "_id": "$user_id",
                    "count": {"$sum": 1}
                }
            },
            {
                "$sort": {"count": -1}
            },
            {
                "$limit": 3
            }
        ]
        
        cursor = db.reviews.aggregate(pipeline)
        results = await cursor.to_list(length=3)
        
        for result in results:
            user_id = result["_id"]
            user = await db.users.find_one({"_id": ObjectId(user_id)})
            if user:
                print(f"Usuário: {user['username']}")
                print(f"Avaliações: {result['count']}")
                print()
        
        # 3. Gêneros mais populares
        print("\nGêneros mais populares:")
        pipeline = [
            {"$unwind": "$genres"},
            {
                "$group": {
                    "_id": "$genres",
                    "count": {"$sum": 1}
                }
            },
            {
                "$sort": {"count": -1}
            },
            {
                "$limit": 5
            }
        ]
        
        cursor = db.movies.aggregate(pipeline)
        results = await cursor.to_list(length=5)
        
        for genre in results:
            print(f"Gênero: {genre['_id']}")
            print(f"Filmes: {genre['count']}")
            print()
        
    except Exception as e:
        print(f"Erro ao executar consultas avançadas: {str(e)}")
    finally:
        await close_mongo_connection(client)

# Executar consultas avançadas
await advanced_queries()

Top 5 filmes com melhor avaliação:

Top 3 usuários mais ativos:
Usuário: bento77
Avaliações: 13

Usuário: manuelacunha
Avaliações: 12

Usuário: isadora58
Avaliações: 12


Gêneros mais populares:
Gênero: Drama
Filmes: 12

Gênero: Crime
Filmes: 9

Gênero: Aventura
Filmes: 8

Gênero: Comédia
Filmes: 7

Gênero: Ação
Filmes: 6

Gênero: Drama
Filmes: 12

Gênero: Crime
Filmes: 9

Gênero: Aventura
Filmes: 8

Gênero: Comédia
Filmes: 7

Gênero: Ação
Filmes: 6



## Conclusão

Os dados de teste foram inseridos com sucesso no MongoDB. Agora você pode usar esse ambiente para testar seu backend FastAPI conectando-se ao banco de dados.

Para executar o backend, certifique-se de que a configuração de conexão do MongoDB esteja correta e inicie sua aplicação com o comando:

```bash
uvicorn main:app --reload
```

Ou usando Docker:

```bash
docker-compose up
```

Observe que este notebook pode ser usado para redefinir os dados de teste quando necessário.

## Criação de Índices

Vamos criar índices estratégicos para otimizar as consultas mais comuns no sistema.

In [ ]:
async def create_indexes():
    """Criar índices estratégicos para otimizar as consultas mais comuns"""
    client, db = await connect_to_mongo()
    
    try:
        print("Criando índices estratégicos...")
        
        # Índices para a coleção movies
        # Índice para busca por título (operações de busca/filtro)
        await db.movies.create_index("title")
        print("✓ Índice criado: movies.title")
        
        # Índice para busca por gêneros (usado nas recomendações e filtros)
        await db.movies.create_index("genres")
        print("✓ Índice criado: movies.genres")
        
        # Índices para a coleção users
        # Índice único para username (login e busca de usuários)
        await db.users.create_index("username", unique=True)
        print("✓ Índice criado: users.username (unique)")
        
        # Índice único para email (registro e validação)
        await db.users.create_index("email", unique=True)
        print("✓ Índice criado: users.email (unique)")
        
        # Índices para a coleção reviews
        # Índice composto para user_id + movie_id (validações de avaliações únicas)
        await db.reviews.create_index(
            [("user_id", 1), ("movie_id", 1)],
            unique=True
        )
        print("✓ Índice criado: reviews.user_id + movie_id (unique)")
        
        # Índice para movie_id (buscar avaliações de um filme)
        await db.reviews.create_index("movie_id")
        print("✓ Índice criado: reviews.movie_id")
        
        print("\nTodos os índices essenciais foram criados com sucesso!")
    except Exception as e:
        print(f"Erro ao criar índices: {str(e)}")
    finally:
        await close_mongo_connection(client)

# Executar criação de índices
await create_indexes()